In [26]:
import numpy as np
import pandas as pd

# import data
admissions = pd.read_csv('binary.csv')

In [27]:
print(admissions[:10])

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4
5      1  760  3.00     2
6      1  560  2.98     1
7      0  400  3.08     2
8      1  540  3.39     3
9      0  700  3.92     2


In [37]:
# Make dummy variables for rank
# one_hot_data = pd.get_dummies(data, columns=['rank'])

data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

data[:10]

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
5,1,760,3.00,0,1,0,0
6,1,560,2.98,1,0,0,0
7,0,400,3.08,0,1,0,0
8,1,540,3.39,0,0,1,0
9,0,700,3.92,0,1,0,0


In [38]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data[field] = (data[field]-mean)/std
#     data.loc[:,field] = (data[field]-mean)/std

data[:10]


,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
5,1,1.491561,-1.024525,0,1,0,0
6,1,-0.239793,-1.077078,1,0,0,0
7,0,-1.624876,-0.814312,0,1,0,0
8,1,-0.412928,0.000263,0,0,1,0
9,0,0.972155,1.392922,0,1,0,0


In [39]:
# Split off random 10% of the data for testing
np.random.seed(42)
#sample is an array of random indexes from the data list.
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
# iloc is the index location. 
data, test_data = data.iloc[sample], data.drop(sample)
len(data)

360

In [40]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

From before we saw that one weight update can be calculated as:


$$ Δw_i = ηδx_i  $$

$$ -(y-\hat{y}) \sigma'(x) $$

where η is the learn rate, and with the error term δ as

$$ δ = (y − \hat{y})f′(h) = (y − \hat{y})f'(∑w_ix_i) $$

Remember, in the above equation $$ -(y-\hat{y}) $$ is the output error, and $$ f′(h) $$
refers to the derivative of the activation function, $$ f(h) $$, for example, for the sigmoid function.

$$ \sigma'(x) = \sigma(x) * (1 - \sigma(x)) $$

We'll call that derivative the output gradient.

Below we'll also be using the sigmoid as the activation function $$ f(h) $$.

In [41]:
#we will use sigmoid
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

In [44]:
# number of records, number of features
n_records, n_features = features.shape
print(features.shape)

(360, 6)


In [46]:
# initialize last_loss
last_loss = None

In [51]:
# Initialize weights random
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)
weights

array([ 0.67923117, -0.30179201, -0.04376196, -0.19849738,  0.65033209,
       -0.17715592])

In [56]:
# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

array([ 0.,  0.,  0.,  0.,  0.,  0.])

In [59]:
# run epoch times
for e in range(epochs):
    # delta weights set to zero each time
    del_w = np.zeros(weights.shape)
    # Loop through all records, x is the input (features), y is the target (target)
    for x, y in zip(features.values, targets):
        

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output (y_hat)
        output = sigmoid(np.dot(x, weights))
        
         # TODO: Calculate the error (y - y_hat)
        error = y - output
        
        # TODO: Calculate the error term (y-y_hat)*f'(output), where sigmoid'(x) = sigmoid(x) * (1- sigmoid)(x) 
        error_term = error * output * (1 - output)
        
        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change = learnrate * error_term * x
        del_w += error_term * x
        
    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records
        
    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss
            
# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.2750032920422348
Train loss:  0.27500329113324257
Train loss:  0.2750032902249705
Train loss:  0.2750032893174182
Train loss:  0.27500328841058425
Train loss:  0.2750032875044686
Train loss:  0.2750032865990699
Train loss:  0.2750032856943877
Train loss:  0.27500328479042097
Train loss:  0.27500328388716905
Prediction accuracy: 0.750
